In [323]:
import pandas as pd
import datetime as dt
import numpy as np

In [324]:
inspections = pd.read_csv('Data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
yelp = pd.read_csv('Data/YELP_ind_CAMIS_merged.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [325]:
inspections.head(3)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,05/11/2018,Violations were cited in the following area(s).,08C,Pesticide use not in accordance with label or ...,Not Critical,5.0,A,05/11/2018,04/28/2019,Cycle Inspection / Initial Inspection
1,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,05/11/2018,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,5.0,A,05/11/2018,04/28/2019,Cycle Inspection / Initial Inspection
2,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462.0,7188924968,Bakery,05/18/2017,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,7.0,A,05/18/2017,04/28/2019,Cycle Inspection / Initial Inspection


In [326]:
inspections.shape

(381911, 18)

In [327]:
inspections[inspections['VIOLATION CODE']=='06D']['GRADE'].value_counts()

A                 13148
B                  1249
C                   310
Z                   134
Not Yet Graded      130
P                    71
Name: GRADE, dtype: int64

In [328]:
inspections['GRADE'].value_counts()

A                 152827
B                  24214
C                   7927
Z                   2559
Not Yet Graded      2535
P                   2284
G                      5
Name: GRADE, dtype: int64

In [329]:
inspections['GRADE'].replace({'Not Yet Graded':np.nan}, inplace=True)
inspections['GRADE'].value_counts()

A    152827
B     24214
C      7927
Z      2559
P      2284
G         5
Name: GRADE, dtype: int64

In [330]:
inspections['CRITICAL FLAG'].value_counts()

Critical          208533
Not Critical      164991
Not Applicable      8387
Name: CRITICAL FLAG, dtype: int64

In [331]:
inspections['CRITICAL FLAG'].replace({'Not Applicable':np.nan}, inplace=True)
inspections['CRITICAL FLAG'].value_counts()

Critical        208533
Not Critical    164991
Name: CRITICAL FLAG, dtype: int64

In [332]:
inspections.isna().sum()

CAMIS                         0
DBA                         509
BORO                          0
BUILDING                    228
STREET                        1
ZIPCODE                    5551
PHONE                         9
CUISINE DESCRIPTION           0
INSPECTION DATE               0
ACTION                     1302
VIOLATION CODE             5785
VIOLATION DESCRIPTION      8387
CRITICAL FLAG              8387
SCORE                     18184
GRADE                    192095
GRADE DATE               192091
RECORD DATE                   0
INSPECTION TYPE            1302
dtype: int64

In [333]:
cols = ['CAMIS', 'DBA', 'CUISINE DESCRIPTION', 'VIOLATION CODE', 'CRITICAL FLAG', 'GRADE', 'GRADE DATE']
insp = inspections[cols]
insp.head(5)

,CAMIS,DBA,CUISINE DESCRIPTION,VIOLATION CODE,CRITICAL FLAG,GRADE,GRADE DATE
0,30075445,MORRIS PARK BAKE SHOP,Bakery,08C,Not Critical,A,05/11/2018
1,30075445,MORRIS PARK BAKE SHOP,Bakery,10F,Not Critical,A,05/11/2018
2,30075445,MORRIS PARK BAKE SHOP,Bakery,06D,Critical,A,05/18/2017
3,30075445,MORRIS PARK BAKE SHOP,Bakery,10F,Not Critical,A,05/18/2017
4,30075445,MORRIS PARK BAKE SHOP,Bakery,04L,Critical,A,02/18/2016


In [334]:
insp.isna().sum()

CAMIS                       0
DBA                       509
CUISINE DESCRIPTION         0
VIOLATION CODE           5785
CRITICAL FLAG            8387
GRADE                  192095
GRADE DATE             192091
dtype: int64

In [335]:
insp = insp.dropna()
insp.shape[0]/insp.shape[0]

1.0

In [336]:
insp.isna().sum()

CAMIS                  0
DBA                    0
CUISINE DESCRIPTION    0
VIOLATION CODE         0
CRITICAL FLAG          0
GRADE                  0
GRADE DATE             0
dtype: int64

In [337]:
# most_recent_inspect = inspections.sort_values('GRADE DATE').drop_duplicates('CAMIS',keep='last')

In [338]:
insp['CRITICAL FLAG'].replace(['Not Critical','Critical'],[0,1],inplace=True)

In [339]:
insp['CRITICAL FLAG'].astype(int)
insp['CRITICAL FLAG'].value_counts()

1    96194
0    92123
Name: CRITICAL FLAG, dtype: int64

In [340]:
# inspections['GRADE DATE'] = inspections['GRADE DATE'].dt.strftime('%m/%d/%Y')
# pd.to_datetime(inspections['GRADE DATE'], format='%m/%d/%Y' ).dt.date.value_counts().rename('VIO_COUNT').to_frame()
# inspections['Vio_Counts']=inspections.groupby(pd.to_datetime(inspections['GRADE DATE'], format='%m/%d/%Y').dt.date.value_counts())
insp['VIO_COUNT'] = insp.groupby(['CAMIS', 'GRADE DATE'])['VIOLATION CODE'].transform('size')
insp.head(2)

,CAMIS,DBA,CUISINE DESCRIPTION,VIOLATION CODE,CRITICAL FLAG,GRADE,GRADE DATE,VIO_COUNT
0,30075445,MORRIS PARK BAKE SHOP,Bakery,08C,0,A,05/11/2018,2
1,30075445,MORRIS PARK BAKE SHOP,Bakery,10F,0,A,05/11/2018,2


In [341]:
insp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188317 entries, 0 to 381484
Data columns (total 8 columns):
CAMIS                  188317 non-null int64
DBA                    188317 non-null object
CUISINE DESCRIPTION    188317 non-null object
VIOLATION CODE         188317 non-null object
CRITICAL FLAG          188317 non-null int64
GRADE                  188317 non-null object
GRADE DATE             188317 non-null object
VIO_COUNT              188317 non-null int64
dtypes: int64(3), object(5)
memory usage: 12.9+ MB


In [342]:
grade_merged = insp.groupby(['CAMIS','GRADE DATE'])['GRADE'].apply(set).reset_index()
grade_merged.rename(columns={'GRADE':'GRADE_LIST'},inplace=True)
grade_merged['GRADE_LIST'].value_counts()
# This is to say, the grade is assigned once at a time.

{A}    70907
{B}     6512
{C}     1856
{P}     1344
{Z}      690
{G}        2
Name: GRADE_LIST, dtype: int64

In [343]:
vio_merged = insp.groupby(['CAMIS','GRADE DATE'])['VIOLATION CODE'].apply(set).reset_index()
vio_merged.rename(columns={'VIOLATION CODE':'VIO_CODE_LIST'},inplace=True)
vio_merged.head(2)

,CAMIS,GRADE DATE,VIO_CODE_LIST
0,30075445,02/18/2016,"{04L, 08A}"
1,30075445,05/11/2018,"{08C, 10F}"


In [344]:
vio_critical = insp.groupby(['CAMIS','GRADE DATE'])['CRITICAL FLAG'].sum().reset_index()
vio_critical.rename(columns={'CRITICAL FLAG':'CRITICAL_COUNT'},inplace=True)
vio_critical.head(2)

,CAMIS,GRADE DATE,CRITICAL_COUNT
0,30075445,02/18/2016,1
1,30075445,05/11/2018,0


In [345]:
vio = pd.merge(vio_merged, vio_critical, on=['CAMIS', 'GRADE DATE'])
vio.head(5)

,CAMIS,GRADE DATE,VIO_CODE_LIST,CRITICAL_COUNT
0,30075445,02/18/2016,"{04L, 08A}",1
1,30075445,05/11/2018,"{08C, 10F}",0
2,30075445,05/18/2017,"{06D, 10F}",1
3,30112340,03/04/2019,"{10B, 02G, 10F}",1
4,30112340,03/13/2018,"{04L, 10B, 08A}",1


In [346]:
insp_ = pd.merge(insp.loc[:, ['CAMIS','DBA','CUISINE DESCRIPTION', 'GRADE DATE', 'GRADE', 'VIO_COUNT']], vio, on=['CAMIS', 'GRADE DATE'])
insp_.head(5)

,CAMIS,DBA,CUISINE DESCRIPTION,GRADE DATE,GRADE,VIO_COUNT,VIO_CODE_LIST,CRITICAL_COUNT
0,30075445,MORRIS PARK BAKE SHOP,Bakery,05/11/2018,A,2,"{08C, 10F}",0
1,30075445,MORRIS PARK BAKE SHOP,Bakery,05/11/2018,A,2,"{08C, 10F}",0
2,30075445,MORRIS PARK BAKE SHOP,Bakery,05/18/2017,A,2,"{06D, 10F}",1
3,30075445,MORRIS PARK BAKE SHOP,Bakery,05/18/2017,A,2,"{06D, 10F}",1
4,30075445,MORRIS PARK BAKE SHOP,Bakery,02/18/2016,A,2,"{04L, 08A}",1


In [347]:
unique_insp = insp_.sort_values('GRADE DATE').drop_duplicates(['CAMIS','DBA'], keep='last').sort_index()
unique_insp

,CAMIS,DBA,CUISINE DESCRIPTION,GRADE DATE,GRADE,VIO_COUNT,VIO_CODE_LIST,CRITICAL_COUNT
3,30075445,MORRIS PARK BAKE SHOP,Bakery,05/18/2017,A,2,"{06D, 10F}",1
14,30112340,WENDY'S,Hamburgers,10/27/2016,A,2,"{04N, 08A}",1
24,30191841,DJ REYNOLDS PUB AND RESTAURANT,Irish,12/14/2017,A,2,"{04H, 10F}",1
29,40356018,RIVIERA CATERERS,American,10/30/2018,A,2,"{06D, 10F}",1
36,40356151,BRUNOS ON THE BOULEVARD,American,08/19/2017,A,2,"{06C, 10F}",1
42,40356483,WILKEN'S FINE FOOD,Delicatessen,06/03/2017,A,3,"{04L, 09A, 08A}",1
56,40356731,TASTE THE TROPICS ICE CREAM,"Ice Cream, Gelato, Yogurt, Ices",06/26/2015,A,2,"{10B, 10F}",0
60,40357217,WILD ASIA,American,06/14/2017,A,1,{10F},0
63,40359480,1 EAST 66TH STREET KITCHEN,American,09/28/2018,A,2,"{06D, 10F}",1
76,40359705,NATHAN'S FAMOUS,Hotdogs,07/08/2016,A,2,"{10A, 02B}",1
